Importing the required Modules and Libraries

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import collections
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer  
from nltk.corpus import stopwords  
from nltk import word_tokenize 
import pickle
import sys
from gensim.models import word2vec 
import gensim
import gensim.downloader as api

C:\Users\devil\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Using  pandas to read the contents of the data-set

In [2]:
txt_data = pd.read_csv("D:/abcd/precisly/Text_Similarity_Dataset.csv") #Location of dataset in my machine
print("Data shape : ", txt_data.shape)
txt_data.head(5)

Data shape :  (4023, 3)


,Unique_ID,text1,text2
0,0,savvy searchers fail to spot ads internet sear...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions to miss out on the net by 2025 40% o...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short by ginepri fifteen-year-...,ruddock backs yapp s credentials wales coach m...
3,3,diageo to buy us wine firm diageo the world s...,mci shares climb on takeover bid shares in us ...
4,4,be careful how you code a new european directi...,media gadgets get moving pocket-sized devices ...


Now we will check for any null values are present in the dataset

In [3]:
txt_data.isnull().sum()

Unique_ID    0
text1        0
text2        0
dtype: int64

Preprocessing the values: removing special symbols, stopwords,lower case words,using lemmatizer from tokenizer function in the notebook.

In [4]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not"  , phrase)
    phrase = re.sub(r"\'re", " are"  , phrase)
    phrase = re.sub(r"\'s",  " is"   , phrase)
    phrase = re.sub(r"\'d",  " would", phrase)
    phrase = re.sub(r"\'ll", " will" , phrase)
    phrase = re.sub(r"\'t",  " not"  , phrase)
    phrase = re.sub(r"\'ve", " have" , phrase)
    phrase = re.sub(r"\'m",  " am"   , phrase)
    return phrase

This snippet would take some time 

In [5]:
preprocessed_text1 = []

# tqdm is for printing the status bar

for sentence in tqdm(txt_data['text1'].values):
    sent = decontracted(sentence)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)

    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text1.append(sent.lower().strip())

100%|██████████| 4023/4023 [10:34<00:00,  6.34it/s]


Meging our preprocessed text 

In [6]:
txt_data['text1'] = preprocessed_text1
txt_data.head(5)

,Unique_ID,text1,text2
0,0,savvy searchers fail spot ads internet search ...,newcastle 2-1 bolton kieron dyer smashed home ...
1,1,millions miss net 2025 40 uk population still ...,nasdaq planning $100m share sale the owner of ...
2,2,young debut cut short ginepri fifteen year old...,ruddock backs yapp s credentials wales coach m...
3,3,diageo buy us wine firm diageo world biggest s...,mci shares climb on takeover bid shares in us ...
4,4,careful code new european directive could put ...,media gadgets get moving pocket-sized devices ...


In [7]:
from tqdm import tqdm
preprocessed_text2 = []

# tqdm is for printing the status bar
for sentance in tqdm(txt_data['text2'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    sent = ' '.join(e for e in sent.split() if e not in stopwords.words('english'))
    preprocessed_text2.append(sent.lower().strip())

100%|██████████| 4023/4023 [27:43<00:00,  2.42it/s]  


Merging our preprocessed text

In [8]:
txt_data['text2'] = preprocessed_text2

txt_data.head(5)

,Unique_ID,text1,text2
0,0,savvy searchers fail spot ads internet search ...,newcastle 2 1 bolton kieron dyer smashed home ...
1,1,millions miss net 2025 40 uk population still ...,nasdaq planning 100m share sale owner technolo...
2,2,young debut cut short ginepri fifteen year old...,ruddock backs yapp credentials wales coach mik...
3,3,diageo buy us wine firm diageo world biggest s...,mci shares climb takeover bid shares us phone ...
4,4,careful code new european directive could put ...,media gadgets get moving pocket sized devices ...


In [9]:
def tokenizer(text):
            #tokenizes and stems the text
            tokens = word_tokenize(text)
            lemmatizer = WordNetLemmatizer() 
            tokens = [lemmatizer.lemmatize(t) for t in tokens]
            return tokens

Using a pretrained model-word2vec-google-news-300.gz, need to download it first

In [10]:
wordmodelfile="D:/abcd/precisly/word2vec-google-news-300/word2vec-google-news-300.gz"
wordmodel= gensim.models.KeyedVectors.load_word2vec_format(wordmodelfile, binary=True)

# This snippet would take a long time, outputs are to confirm if the kernel is working smoothly.
It checks if the words are present in our model.If they are present, it calculates it similarity score, and if not it removes them

In [13]:
from tqdm import tqdm
#A list to store similarity score
similarity = [] 



for ind in txt_data.index:
    
        s1 = txt_data['text1'][ind]
        s2 = txt_data['text2'][ind]
        
        if s1==s2:
                 similarity.append(0.0) # 0 means highly similar
                
        else:   

            s1words = tokenizer(s1)
            s2words = tokenizer(s2)

       #the vocabulary considered in the word embeddings
            vocab=wordmodel.index_to_key
            
            if len(s1words and s2words)==0:
                    similarity.append(1.0)

            else:
                
                for word in s1words.copy(): 
                    #To remove all words which are not found in the vocab variable
                    if (word not in vocab):
                           
                            print(word)
                            s1words.remove(word)
                        
                    
                for word in s2words.copy(): #idem

                    if (word not in vocab):
                            print(word)
                            s2words.remove(word)
                            
                similarity.append((1-wordmodel.n_similarity(s1words, s2words)))

87
84
jeeves
50
43
34
36
29
44
44
48
macklin
jeeves
17
33
68
62
18
macklin
kieron
10
bowyer
stelios
giannakopoulos
jussi
jaaskelainen
35
bowyer
giannakopoulos
okocha
davy
davy
giannakopoulos
hierro
69th
jaaskelainen
allardyce
graeme
souness
45
boumsong
babayaro
faye
bowyer
77
ameobi
bowyer
35
69
jaaskelainen
fadiga
14
gotty
haim
giannakopoulos
okocha
vaz
77
hierro
64
davy
jaidi
poole
haim
hierro
giannakopoulos
41
50
430
gloucestershire
2025
40
23
briton
27
50
briton
hosford
hosford
16
2025
organisation
hosford
hosford
100m
100m
52m
1990s
2000
2001
2002
1990s
ginepri
robby
ginepri
200
14
10
13
ginepri
ginepri
gambill
hernych
dancevic
carlsen
irakli
labadze
defence
goldstein
agassi
agassi
ruddock
yapp
ruddock
yapp
21
ruddock
ruddock
yapp
defence
ruddock
yapp
gough
heineken
ruddock
recognised
diageo
diageo
chalone
260m
134m
diageo
smirnoff
diageo
diageo
2005
chalone
diageo
favourable
diageo
menezes
diageo
89bn
28bn
31
2003
chalone
69
mci
mci
mci
mci
mci
worldcom
20
15
mci
mci
mci
16bn
mci

26
kinnock
kinnock
labour
kinnock
bedwellty
1969
2018
10
1970s
1950s
defence
2000
defence
favour
briton
2005
75
30
53
32
14
36
2003
1960s
nesbit
1991
favourite
1980s
1960s
2004
minogue
60
papamichael
jolin
42m
22m
21m
11m
favourite
garfield
13
1974
yaffle
marvellous
1977
wombling
11
catweazle
gandalf
thundercats
keanu
gere
pandy
grearest
thundercats
15
dougal
disneyfying
programme
realised
humour
pugwash
programme
pludering
20
saphire
galactica
70
80
10
15
mungo
connery
plany
programme
thundercats
favourite
squarepants
chorlton
rentaghost
terrahawks
citreon
rushden
programme
34
postgate
fingerbobs
yoffi
fingermouse
krusty
90
supermarionation
programme
bagpuss
chorton
gurgitation
starsky
chorlton
27
chorlton
aaliyah
aaliyah
2001
blackground
22
2003
aaliyah
blackground
penski
700
320kg
aaliyah
honoured
usdaw
whitefield
whitefield
usdaw
hannett
500
ukip
1992
viktor
yushchenko
levelled
ukip
ukip
knapman
ukip
11
organisation
ukip
organisation
ukip
kilroy
ukip
ukip
ukip
kilroy
kilroy
ukip
20

2003
25
22
10
lansdowne
27
1500
bbc1
1948
13
arcy
driscoll
tindall
vickery
centre
mathew
tait
hodgson
twickenham
1987
dempsey
driscoll
horgan
hickie
corrigan
byrne
connell
easterby
sheahan
horan
callaghan
easterby
humphreys
maggs
cueto
barkley
lewsey
hodgson
elli
rowntree
grewcock
worsley
corry
titterrell
borthwick
hazell
goode
wraz
182
100
000
52
000
atwood
commercialised
programme
100
000
52
590
wraz
schenck
wraz
14
23
000
22
13
44
34
48
41
47
36
22
953
globescan
louisa
untrammelled
willey
2001
15
2004
2005
22
15
2004
2005
labour
gateshead
60
65
23
labour
800
000
serwotka
20th
800
000
70
000
20
000
23
261
135
30
179
11
2003
beaverton
favourable
2004
2005
2004
armand
leroi
10
000
armistead
recognises
kunzru
eyre
alexei
sayle
susanna
norrell
19th
armand
leroi
armistead
eyre
leroi
zadie
2000
ormond
aynsley
100
000
aynsley
aynsley
aynsley
13
2005
11
488
76
2005
13
000
naoki
kamiyama
2003
280bn
280bn
155bn
1999
altria
lorillard
liggett
280bn
50
206bn
46
1998
altria
hingis
martina
hingis
p

safin
marat
safin
safin
safin
lundgren
25
sampras
2000
safin
2000
sampras
favourite
johansson
2002
lleyton
safin
25
2000
200m
musselburgh
31
1998
briton
moazzam
begg
mubanga
belmar
feroz
abbasi
2003
2000
ancram
menzies
azmat
begg
moazzam
abbasi
mubanga
moazzam
begg
labour
godsiff
begg
godsiff
chakrabarti
practise
12
belmarsh
woodhill
12
48
mamdouh
habib
offence
140
labour
1997
auschwitz
fulfil
labour
60m
100m
persson
luca
verdecchia
61
62
60m
39secs
1998
1500m
800m
olga
fedorova
kallur
60m
23secs
maduaka
200m
31
4x200m
63
48
22
16
87m
tosin
15
80m
65m
buckfield
51cm
800m
400m
200m
djhone
400m
60m
irina
shevchenko
claxton
1500m
200
metre
4x200m
ageing
101
torao
toshitsune
toshitsune
23
000
13
114
osaka
toshitsune
toshitsune
85
78
83
yuji
shimizu
shimizu
2007
127
100
30
2050
60
30
shigeo
morioka
centre
140
industrialised
2010
harbour
elli
2050
2030
2010
46
420
130
65s
labour
ageing
2020
industrialised
toshitsune
101
ukip
ukip
anglian
ukip
woodbridge
ukip
whitehall
12
500
800
axe
104
000


2004
pbgc
1974
pbgc
jolanda
ceplak
medallist
800m
ceplak
ceplak
briton
slovenian
800m
2002
ceplak
1500m
2003
ceplak
ceplak
erfurt
60m
erfurt
leipzig
400m
mackie
isinbayeva
yelena
isinbayeva
89
metre
lievin
12th
88m
05m
60m
100m
46secs
50
50
50
97
50
50
50
jadakiss
97
50
50
50
50
24
50
50
50
50
shakur
1996
53
metrovias
centre
shanklin
centre
charvis
ruddock
henson
centre
11
henson
ruddock
henson
ronan
19
13
munster
10
14
byrne
byrne
15
driscoll
cueto
driscoll
favourite
favourite
geordan
hickie
26
1600
bbc1
1994
1999
2001
38
gwent
luscombe
ruddock
ruddock
laporte
yannick
nyanga
julien
laharrague
aurelien
rougerie
yannick
jauzion
criticised
fabien
pelous
laharrague
rougerie
jauzion
traille
dominici
delaigue
yachvili
marconnet
pelous
thion
betsen
nyanga
bonnaire
bourgoin
servat
milloud
lamboley
harinordoquy
mignoni
michalak
grandclaude
shanklin
henson
davy
cockbain
sidoli
mcbryde
yapp
sowden
30
angola
programme
luanda
gera
programme
programme
angola
30
2002
10
90
modelled
lapete
28
angola


...

350m
wpp
ogilvy
mather
cordiant
rubicam
wpp
2004
favourable
wpp
khodorkovsky
yukos
mikhail
khodorkovsky
yukos
khodorkovsky
59
menatep
yukos
leonid
nevzlin
yukos
nevzlin
khodorkovsky
yukos
yuganskneftegaz
yuganskneftegaz
rosneft
yuganskneftegaz
khodorkovsky
nevzlin
yukos
yuganskneftegaz
yukos
totalling
27bn
15bn
khodorkovsky
2003
vladimir
khodorkovsky
44
205
110m
142m
92
103
05
935
648
33
34
2005
janco
pyykkonen
2004
778
27bn
2004
2005
2005
35bn
48
52
2005
37bn
62
18
11
aadc
250
aadc
250
150
100
aadc
specialised
buckingham
35mm
500
2048
1080
35mm
lomu
lomu
29
2004
twickenham
15
lomu
lomu
lomu
1994
lomu
honour
10
titus
kluivert
31
mellor
baros
bowyer
offence
kluivert
baros
20
25
souness
11
kluivert
kieron
bowyer
hyypia
37
baros
mellor
xabi
kewell
baros
jermaine
ameobi
bowyer
florent
sinama
pongolle
dudek
finnan
hyypia
carragher
riise
nunez
73
kewell
traore
85
baros
mellor
sinama
pongolle
75
hamann
35
mellor
38
baros
61
brien
bowyer
80
zogbia
72
kluivert
58
ameobi
bowyer
77
bowyer
kluiver

maury
2001
2002
centre
lacklustre
2001
367
000
2001
castaignede
laporte
castaignede
laporte
castaignede
laporte
lansdowne
45
hodgson
barkley
twickenham
castaignede
laporte
laporte
criticised
defence
castaignede
12
castaignede
43
castaignede
30
yann
delaigue
frederic
michalak
10
delaigue
michalak
castaignede
beverley
2003
charlene
elli
18
letisha
17
1997
10
500
offence
70
2003
2004
2003
25
22
10
centred
niven
ringworld
errera
capitalised
16
errera
600
000
15
2001
10
10
errera
errera
analysed
errera
11
icann
10
icann
23
000
10
250
jarvis
jarvis
16
33
ferrovial
146m
281m
jarvis
230m
2005
2006
14
pfi
jarvis
ferrovial
33
piccadilly
amey
2003
jarvis
100m
2005
jarvis
lovell
jarvis
16
18
mcclaren
middlesbrough
mcclaren
128
mcclaren
mcclaren
mcclaren
mcclaren
equaliser
favourite
middlesbrough
boudewijn
zenden
zenden
cancelling
2004
cancelled
30
22
82
recognised
federline
portishead
portishead
portishead
oxfam
19
1995
portishead
portishead
favourite
1994
portishead
organisation
endeavour
2005
ta

marat
safin
safin
safin
lundgren
25
sampras
2000
safin
2000
sampras
favourite
johansson
2002
lleyton
safin
25
2000
favour
labour
programme
criticised
59
2001
swivelled
travelling
obermann
3gsm
100
000
mdas
20
38
12
300
20
000
16
mcbride
gner
obermann
3gsm
14
17
yukos
gref
kommersant
yukos
28
yuganskneftegaz
60
yukos
10bn
baikal
baikal
rosneft
gazprom
andrei
illarionov
vladimir
gref
nationalisation
iliaronov
privatisation
yukos
1990s
rosneft
yuganskneftegaz
yukos
nationalise
gref
yukos
gref
kommersant
spurlock
spurlock
spurlock
organiser
honour
spurlock
goldwyn
spurlock
25lb
spurlock
spurlock
mclibel
1990s
60
000
40
000
centre
2000
centre
centre
bertie
ahern
centre
10
13
dempsey
ahern
14
iband
devoices
100
50
16
10
2003
56
niro
niro
22m
11
niro
fockers
niro
18m
20th
eastwood
11
11
15th
35m
17m
25m
quaid
johansson
13
centre
11
twickenham
corry
mathew
tait
barkley
tait
centre
tait
tait
tait
henson
22
henson
centre
hazell
1980s
60
angadias
shailendra
1985
favourite
dougray
clive
870
39
mcg

rohm
sonaptic
monteith
alastair
sibbald
sonaptic
montieth
sonaptic
sonaptic
10
15
montieth
moya
moya
paradorn
srichaphan
28
000
srichaphan
2003
moya
moya
26
150
000
scrichapan
000
25
000
40
000
25
40
berson
50
1700
97p
87
44p
85
500m
258
everasia
khurmala
iog
himrin
ironhorse
khurmala
100
000
himrin
suba
luhais
jugnot
jugnot
depardieu
2004
jugnot
45m
77m
figaro
choristes
jugnot
1949
2004
depardieu
35m
31m
depardieu
figaro
jugnot
2004
55m
45m
rwanda
figaro
audrey
tautou
10th
885
000
611
000
jeunet
fiancailles
outkast
outkast
outkast
outkast
speakerboxxx
3000
2003
3000
outkast
d12
anastacia
outkast
favourite
timberlake
d12
d12
anastacia
lavigne
anastasia
lavigne
knowles
linkin
bennington
honour
franz
natasha
bedingfield
jamelia
11
12
11th
xzibit
000
stefani
pharrell
organised
anastacia
200
000
timberlake
400
05
idvd
centre
programme
80gb
centre
80gb
centre
80gb
120gb
centre
centre
barlow
itx
12
700
sridharan
lightpointe
25gbps
000
512kbps
sridharan
12
70
000
sridharan
trowbridge
chicoine

nyanga
233
ziers
2003
marat
safin
lleyton
safin
molik
10
10
21
baltacha
120
65
2005
1994
15m
honour
riddick
2004
2003
200
000
100
152m
2003
theatre
theatre
griffith
festen
theatre
lenkiewicz
redmayne
rhys
shakespearean
theatre
oram
whatsonstage
11
000
theatre
rigg
judi
dench
revolutionised
1922
18
20
grey
40
1961
romitaand
northstar
50
1971
1999
2001
programme
12
40
2002
2004
857m
bosvelt
bosvelt
34
bosvelt
feyenoord
2003
keegan
realised
keegan
ferrero
robredo
seville
moya
jordi
arrese
18
ferrero
26
robredo
ferrero
arrese
favour
harmonise
lichtenberg
1948
19
favour
2000
gleeson
1983
24
londoner
damien
cudicini
cudicini
celestine
babayaro
deputised
ferreira
hudell
tillen
11
arjen
odpm
180
226
180
444
2005
13
odpm
hbos
odpm
15
2004
2005
stabilises
odpm
2003
odpm
192
713
139
544
116
542
111
314
odpm
26
humber
21
21
15
262
825
balco
balco
ilston
defence
balco
valente
korchemny
balco
centre
usada
thg
usada
10
thg
alvin
balco
100m
dwain
thg
2003
goldfine
18
goldfine
goldfine
goldfine
goldfin

Making a dataframe with coloumns: similarity score, unique id

In [14]:
final_output = pd.DataFrame({'Unique_ID':txt_data.Unique_ID,
                     'Similarity_score':similarity})
final_output.head(5)

,Unique_ID,Similarity_score
0,0,0.389470
1,1,0.292066
2,2,0.272890
3,3,0.184955
4,4,0.171677


Saving the final output from our dataframe into a .csv file

In [ ]:
final_output.to_csv('final_output.csv',index=False)